In [1]:
from database import database
from matplotlib import pyplot as plt
import sklearn.feature_extraction.image
from sklearn.feature_extraction.image import extract_patches
import tables
import random
from SRCNN import run
from build_model import build_model
import tensorflow as tf

In [2]:
random.seed(31415926)
args = {}
args['filedir'] = 'train_data'
args['pattern'] = '*.bmp'
args['database_name'] = 'mock'
args['export_dir'] = 'export'
args['patch_shape'] = (64,64,3)
args['stride_size'] = 32



data_set = database(**args)
data_set.initialize()

In [3]:
run(data_set, num_epoch = 2, batch_size = 13)

Preparing data...
Instructions for updating:
Use `tf.global_variables_initializer` instead.


100%|████████████████████████████████████████| 226/226 [00:08<00:00, 25.95it/s]


EPOCH: 0 Avg Loss: 0.1355726443214622


100%|████████████████████████████████████████| 226/226 [00:07<00:00, 30.43it/s]


EPOCH: 1 Avg Loss: 0.0479210277862715


In [6]:
print(data_set.size('train'))

2946


In [7]:
img_size,label_size=data_set.peek('train')
print(img_size,img_size)

(2946, 64, 64, 3) (2946, 64, 64, 3)
